In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, UpSampling2D
from tensorflow.keras.models import Model
from transformers import pipeline
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Dataset path
DATASET_PATH = '/kaggle/input/interior-design/resized_images'
IMG_SIZE = 128  # Standardize image size
EPOCHS = 200
BATCH_SIZE = 32
SAVE_PATH = "/kaggle/working/interior_design_gan_model.h5"

In [ ]:
# Helper function to load images
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img = img_to_array(img) / 255.0
        images.append(img)
    return np.array(images)

# Load dataset
print("Loading dataset...")
images = load_images(DATASET_PATH)
print(f"Loaded {images.shape[0]} images with shape {images.shape[1:]}")

In [ ]:
# Generator model
def build_generator():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(inputs)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    outputs = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    return Model(inputs, outputs)

# Discriminator model
def build_discriminator():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = Conv2D(64, (3, 3), padding='same', strides=(2, 2))(inputs)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)

# Build GAN
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Combined GAN model
discriminator.trainable = False
gan_input = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

print("Generator and Discriminator models built.")
generator.summary()
discriminator.summary()

In [ ]:
# Function to train GAN
def train_gan(generator, discriminator, gan, images, epochs, batch_size):
    for epoch in range(epochs):
        idx = np.random.randint(0, images.shape[0], batch_size)
        real_images = images[idx]
        fake_images = generator.predict(real_images)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        g_loss = gan.train_on_batch(real_images, np.ones((batch_size, 1)))

        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
            plot_images(generator.predict(images[:5]), title="Generated Images")

# Train the model
train_gan(generator, discriminator, gan, images, EPOCHS, BATCH_SIZE)

In [ ]:
# NLP Model for Text-to-Image
text_to_image = pipeline("text-to-image-generation")

def generate_image_from_text(prompt):
    print(f"Generating image for prompt: {prompt}")
    results = text_to_image(prompt)
    image_data = results[0]['image']
    plt.imshow(image_data)
    plt.axis('off')
    plt.title(f"Generated for: {prompt}")
    plt.show()

# Example
generate_image_from_text("A cozy room with a fireplace and a modern couch.")

In [ ]:
# Upload Widget
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
display(upload_widget)

def on_image_upload(change):
    if upload_widget.value:
        file_info = next(iter(upload_widget.value.values()))
        img_data = file_info['content']
        with open("uploaded_image.jpg", "wb") as f:
            f.write(img_data)
        print("Image uploaded successfully!")

upload_widget.observe(on_image_upload, names='value')

# User Preferences
room_style = widgets.Dropdown(options=['Modern', 'Vintage', 'Industrial'], description='Style:')
room_color = widgets.ColorPicker(value='#ffffff', description='Color:')
display(room_style, room_color)

# Generate Design
uploaded_image_path = "uploaded_image.jpg"

def generate_design_based_on_preferences():
    if os.path.exists(uploaded_image_path):
        img = cv2.imread(uploaded_image_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
        img = np.expand_dims(img, axis=0)
        styled_img = generator.predict(img)[0]
        plt.imshow(styled_img)
        plt.axis('off')
        plt.title(f"Design in {room_style.value} Style with Color {room_color.value}")
        plt.show()
    else:
        print("Upload an image to proceed.")

generate_design_based_on_preferences()

In [ ]:
# Save Generator model
generator.save(SAVE_PATH)
print(f"Generator model saved to {SAVE_PATH}")